In [61]:
#basics
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math
import os


In [62]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [63]:
input_path = None

In [64]:
output_path = None

In [65]:
if __name__ == '__main__':
    INPUT_PATH = os.getenv('INPUT_PATH')
    OUTPUT_PATH = os.getenv('OUTPUT_PATH')

    if not INPUT_PATH or not OUTPUT_PATH:
        raise ValueError("Both INPUT_PATH and OUTPUT_PATH environment variables must be set")
    
    if (INPUT_PATH == None):
        input_path_df = 'input/test_dataset.csv'
    else:
        input_path_df = INPUT_PATH
    output_path_df = 'preprocessed.csv'


In [66]:
df = pd.read_csv(input_path_df)
df.head()

,company_id,industry_id,topic_id,category_id,week,interest_level
0,228,88,110,7,8,18
1,228,88,63,6,8,18
2,228,88,18,12,8,25
3,228,88,30,12,8,23
4,228,88,42,12,8,18


# Task №1: Proof-of-Concept stage


1. Basic exploratory analysis of the dataset

In [67]:
print("Shape of the dataset:", df.shape)
print("")
print("Data types:")
print(df.dtypes)

Shape of the dataset: (1641482, 6)

Data types:
company_id        int64
industry_id       int64
topic_id          int64
category_id       int64
week              int64
interest_level    int64
dtype: object


In [68]:
print("unique number of values for company_id:", df['company_id'].nunique())

unique number of values for company_id: 8717


In [69]:
print("unique number of values for industry_id:", df['industry_id'].nunique())

unique number of values for industry_id: 130


In [70]:
print("unique number of values for topic_id:", df['topic_id'].nunique())

unique number of values for topic_id: 114


In [71]:
print("unique number of values for category_id:", df['category_id'].nunique())

unique number of values for category_id: 14


In [72]:
print(df.isnull().sum())
print("No missing values")

company_id        0
industry_id       0
topic_id          0
category_id       0
week              0
interest_level    0
dtype: int64
No missing values


In [73]:
print("weekly interest levels in mean median and count")
weekly_stats = df.groupby('week')['interest_level'].agg(['mean', 'median', 'count'])
weekly_stats['mean_diff'] = weekly_stats['mean'].diff()
#interest rate changes sligly on each week
print(weekly_stats)

weekly interest levels in mean median and count
           mean  median   count  mean_diff
week                                      
0     22.278214    22.0  108643        NaN
1     23.491262    24.0  113807   1.213048
2     22.953398    24.0  116820  -0.537863
3     22.949783    24.0  119561  -0.003615
4     23.175513    24.0  126344   0.225730
5     22.904185    24.0  129562  -0.271328
6     22.017105    22.0  127039  -0.887080
7     22.255516    22.0  124767   0.238411
8     22.665579    23.0  124681   0.410062
9     22.381244    24.0  137660  -0.284335
10    22.406066    24.0  133131   0.024823
11    22.171088    23.0  139998  -0.234978
12    22.203730    23.0  139469   0.032642


In [74]:
print("weekly interest levels for each category in mean median and count")
weekly_stats = df.groupby(['category_id'])['interest_level'].agg(['mean', 'median', 'count'])
weekly_stats['mean_diff'] = weekly_stats['mean'].diff()
print(weekly_stats.head(13))

weekly interest levels for each category in mean median and count
                  mean  median   count  mean_diff
category_id                                      
0            21.982576    22.0   38796        NaN
1            23.408432    24.0   15584   1.425856
2            22.818843    24.0   71220  -0.589589
3            22.101452    23.0   36924  -0.717391
4            23.059242    24.0    2110   0.957790
5            22.405443    23.0   29543  -0.653799
6            22.425333    23.0   79239   0.019891
7            22.016917    23.0   38483  -0.408417
8            23.226448    24.0    8598   1.209531
9            22.795239    24.0   32892  -0.431209
10           22.050557    23.0   83173  -0.744682
11           22.596940    23.0  109877   0.546383
12           22.632560    24.0  919550   0.035620


2. Proposed scoring method

In [75]:
# Week to week

#Original Volatility shows the average standard deviation of the original interest levels 
#across different topics. High volatility means more fluctuations in interest levels 

#Trend Score Volatility represents the average standard deviation of the smoothed trend scores across 
#different topics. low volatility means the trend scores are more stable and less prone to big fluctuations 

In [76]:
df = df.sort_values(by=['topic_id', 'week'])
df.head(10)

,company_id,industry_id,topic_id,category_id,week,interest_level
11834,237,109,0,13,0,24
61639,564,51,0,13,0,24
81077,797,57,0,13,0,24
111301,2903,109,0,13,0,24
112805,2908,38,0,13,0,25
161316,3397,109,0,13,0,25
289322,222,12,0,13,0,24
348190,3727,31,0,13,0,18
362915,3902,109,0,13,0,24
407124,3850,5,0,13,0,18


In [77]:
# find previous interest rate to substract them 
df['prev_interest_level'] = df.groupby(['topic_id'])['interest_level'].shift(1).fillna(0)
df.head(100)

,company_id,industry_id,topic_id,category_id,week,interest_level,prev_interest_level
11834,237,109,0,13,0,24,0.0
61639,564,51,0,13,0,24,24.0
81077,797,57,0,13,0,24,24.0
111301,2903,109,0,13,0,24,24.0
112805,2908,38,0,13,0,25,24.0
161316,3397,109,0,13,0,25,25.0
289322,222,12,0,13,0,24,25.0
348190,3727,31,0,13,0,18,24.0
362915,3902,109,0,13,0,24,18.0
407124,3850,5,0,13,0,18,24.0


In [78]:
# Calculate weekly changes for each topic
df['weekly_change'] = df['interest_level'] - df['prev_interest_level']
df.head(15)

,company_id,industry_id,topic_id,category_id,week,interest_level,prev_interest_level,weekly_change
11834,237,109,0,13,0,24,0.0,24.0
61639,564,51,0,13,0,24,24.0,0.0
81077,797,57,0,13,0,24,24.0,0.0
111301,2903,109,0,13,0,24,24.0,0.0
112805,2908,38,0,13,0,25,24.0,1.0
161316,3397,109,0,13,0,25,25.0,0.0
289322,222,12,0,13,0,24,25.0,-1.0
348190,3727,31,0,13,0,18,24.0,-6.0
362915,3902,109,0,13,0,24,18.0,6.0
407124,3850,5,0,13,0,18,24.0,-6.0


Normalization formula is: (X- Xmin) / (Xmax - Xmin) 
But we need Normalization formula for custom ranges 0 - 100, therefore I will normalize data from -1 to 1 and then Transform to Trend Score by Converting the normalized changes to a trend score ranging from 0 to 100.
Normalizing to 0 to 1 and then scaling to 0 to 100 would map min and max but it wouldn't naturally place a stable trend at 50.

In [79]:
# Find the max and min of weekly changes for normalization
max_change = df['weekly_change'].max()
min_change = df['weekly_change'].min()
print('maximum change in levels with previos one', max_change)
print('minimum change in levels with previos one', min_change)

maximum change in levels with previos one 86.0
minimum change in levels with previos one -82.0


In [80]:
# xnormalized for -1 to 1 =(b−a) (X- Xmin) / (Xmax - Xmin) + a
# (b - a) is 1 - (-1) = 2 and a is -1

In [81]:
df['normalized_change'] = (2*(df['weekly_change'] - min_change)) / (max_change - min_change) - 1
df.head(20) # normalized data from -1 to 1

,company_id,industry_id,topic_id,category_id,week,interest_level,prev_interest_level,weekly_change,normalized_change
11834,237,109,0,13,0,24,0.0,24.0,0.261905
61639,564,51,0,13,0,24,24.0,0.0,-0.023810
81077,797,57,0,13,0,24,24.0,0.0,-0.023810
111301,2903,109,0,13,0,24,24.0,0.0,-0.023810
112805,2908,38,0,13,0,25,24.0,1.0,-0.011905
161316,3397,109,0,13,0,25,25.0,0.0,-0.023810
289322,222,12,0,13,0,24,25.0,-1.0,-0.035714
348190,3727,31,0,13,0,18,24.0,-6.0,-0.095238
362915,3902,109,0,13,0,24,18.0,6.0,0.047619
407124,3850,5,0,13,0,18,24.0,-6.0,-0.095238


In [82]:
df['trend_score'] = (df['normalized_change'] + 1) * 50
df.head()   # get trend scores between 0 and 100
# By adding 1 to df['normalized_change'], 
# I shift the range from [-1, 1] to [0, 2] and multiply by 50 to have range from [0,100]

,company_id,industry_id,topic_id,category_id,week,interest_level,prev_interest_level,weekly_change,normalized_change,trend_score
11834,237,109,0,13,0,24,0.0,24.0,0.261905,63.095238
61639,564,51,0,13,0,24,24.0,0.0,-0.023810,48.809524
81077,797,57,0,13,0,24,24.0,0.0,-0.023810,48.809524
111301,2903,109,0,13,0,24,24.0,0.0,-0.023810,48.809524
112805,2908,38,0,13,0,25,24.0,1.0,-0.011905,49.404762


In [83]:
# I will use smoothing as Original trend scores might contain short-term fluctuations or 
# noise due to variations in weekly interest levels. Smoothing helps to filter out this noise

In [84]:
# Apply smoothing to the trend scores using a rolling average with a window of 3
# Original trend scores might contain short-term fluctuations or noise due to variations in weekly 
# interest levels. Smoothing helps to filter out this noise
# maybe smoothing will help if there is noise
df['smoothed_trend_score'] = df.groupby(['topic_id'])['trend_score'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())
df.head() # smoothing by average or median

,company_id,industry_id,topic_id,category_id,week,interest_level,prev_interest_level,weekly_change,normalized_change,trend_score,smoothed_trend_score
11834,237,109,0,13,0,24,0.0,24.0,0.261905,63.095238,63.095238
61639,564,51,0,13,0,24,24.0,0.0,-0.023810,48.809524,55.952381
81077,797,57,0,13,0,24,24.0,0.0,-0.023810,48.809524,53.571429
111301,2903,109,0,13,0,24,24.0,0.0,-0.023810,48.809524,48.809524
112805,2908,38,0,13,0,25,24.0,1.0,-0.011905,49.404762,49.007937


3. Suggestions on how to assess "the volatility of the scores". Please, design a metric to show how
good is the solution comparing to the baseline (original scores).


In [85]:
# I will measure the volatility of the trend scores
# A common measure for volatility is coefficient of variation (CV)
# Formula for coefficient of Variation (CV) is standard deviation / mean
# Calculate the CV for both baseline and my solution and see the difference
# higher improvement percentage means a greater reduction in volatility

In [86]:
# coefficients of original and smoothed trend scores
original_cv = df.groupby('topic_id')['interest_level'].std() / df.groupby('topic_id')['interest_level'].mean()
smoothed_cv = df.groupby('topic_id')['smoothed_trend_score'].std() / df.groupby('topic_id')['smoothed_trend_score'].mean()

In [87]:
cv_improvement = ((original_cv - smoothed_cv) / original_cv) * 100
cv_improvement # in percentage

topic_id
0      85.581864
1      87.920430
2      87.375763
3      71.070533
4      86.891399
5      87.157364
6      85.879316
7      77.004866
8      86.551911
9      87.322866
10     87.341457
11     87.334016
12     86.753690
13     87.405858
14     88.908395
15     87.639531
16     87.007774
17     86.386632
18     87.648210
19     87.033019
20     86.940834
21     87.910010
22     86.858605
23     87.632367
24     87.114214
25     49.323416
26     86.745518
27     87.781533
28     84.678660
29     86.558631
30     87.140220
31     87.087766
32     87.270780
33     87.018559
34     89.091982
35     46.148932
36     86.424058
37     88.068991
38          -inf
39     87.314547
40     87.648311
41     86.804046
42     87.267108
43     87.225706
44     63.568179
45     87.041135
46     80.249353
47     87.377911
48     88.401747
49     86.683508
50     82.929276
51     87.093029
52     87.700239
53     87.422843
54     86.948382
55     78.459894
56     86.561759
57     87.193188
58   

In [88]:
cv_comparison_df = pd.DataFrame({
    'original_cv': original_cv,
    'smoothed_cv': smoothed_cv,
    'cv_improvement (%)': cv_improvement
})

cv_comparison_df.head()

,original_cv,smoothed_cv,cv_improvement (%)
topic_id,,,
0,0.149463,0.021550,85.581864
1,0.394257,0.047625,87.920430
2,0.240559,0.030369,87.375763
3,0.146863,0.042487,71.070533
4,0.368281,0.048277,86.891399


Saving the trend score for the past 4 weeks

In [89]:
recent_df = df[df['week'] >= df['week'].max() - 3]

# Save the filtered data to a CSV file
recent_df.to_csv(output_path_df, index=False)

In [90]:
recent_df.head()

,company_id,industry_id,topic_id,category_id,week,interest_level,prev_interest_level,weekly_change,normalized_change,trend_score,smoothed_trend_score
12036,237,109,0,13,9,25,22.0,3.0,0.011905,50.595238,50.198413
17680,461,111,0,13,9,18,25.0,-7.0,-0.107143,44.642857,48.809524
61910,564,51,0,13,9,25,18.0,7.0,0.059524,52.976190,49.404762
81213,797,57,0,13,9,25,25.0,0.0,-0.023810,48.809524,48.809524
83205,59,109,0,13,9,18,25.0,-7.0,-0.107143,44.642857,48.809524


Weekly interest levels were computed, revealing slight fluctuations over time. A scoring method was proposed to assess interest trends more effectively. This involved calculating weekly changes in interest levels for each topic, normalizing these changes to a range from -1 to 1, converting them to trend scores from 0 to 100, and applying smoothing to mitigate short-term fluctuations. The volatility of these trend scores was measured using the coefficient of variation (CV). The comparison between the original and smoothed trend scores show a significant reduction in volatility, highlighting the effectiveness of the proposed scoring method in stabilizing interest trends.